In [1]:
import torch
from torch import nn

from data_loading import load_dataset_3d

from core import train, test

In [4]:
BATCH_SIZE = 64

In [5]:
dataset = load_dataset_3d('games_dataset.csv', BATCH_SIZE)
len(dataset)

9117

In [6]:
DATASET_SIZE = 8000

In [7]:
train_dataset = dataset[:DATASET_SIZE]
test_dataset = dataset[DATASET_SIZE:]
len(train_dataset),len(test_dataset)

(8000, 1117)

In [8]:
validation_dataset = load_dataset_3d('puzzle_dataset.csv', BATCH_SIZE)
len(validation_dataset)

727

In [9]:
def accuracy(out, truth):
    return torch.abs(truth - out)

In [10]:
class Model(nn.Module):
    def __init__(self, *args, **kwargs):
        super(Model, self).__init__()
        self.classifier = nn.Sequential(*args, **kwargs)

    def forward(self, X):
        return self.classifier.forward(X)


model = Model(nn.Conv2d(8, 32, kernel_size=4, stride=2, padding=4),
              nn.ReLU(),
              nn.Conv2d(32, 128, kernel_size=6, stride=2, padding=4),
              nn.ReLU(),
              nn.Conv2d(128, 512, kernel_size=6, stride=2, padding=2),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2, stride=1),
              nn.Conv2d(512, 64, kernel_size=1, stride=2),
              nn.ReLU(),
              nn.Flatten(),
              nn.Linear(64, 16),
              nn.ReLU(),
              nn.Linear(16, 1),
              )

In [11]:
train(train_dataset,
      BATCH_SIZE,
      model,
      nn.MSELoss(),
      torch.optim.SGD(model.classifier.parameters(), lr=0.02),
      accuracy,
      60)

Dataset size: 8000
Epoch [1/60], train_loss: 8.046044450700283, train_accuracy: 2.115641266465187, time: 13.483s
Epoch [2/60], train_loss: 5.268648249477148, train_accuracy: 1.6906509106904268, time: 11.913s
Epoch [3/60], train_loss: 4.2897482251822945, train_accuracy: 1.51181783477962, time: 13.314s
Epoch [4/60], train_loss: 3.603443098142743, train_accuracy: 1.3728560545891524, time: 13.482s
Epoch [5/60], train_loss: 3.0720151232779025, train_accuracy: 1.260908000074327, time: 13.447s
Epoch [6/60], train_loss: 2.612149955585599, train_accuracy: 1.1584448065906763, time: 13.353s
Epoch [7/60], train_loss: 2.2324160010516643, train_accuracy: 1.0684453205391764, time: 13.445s
Epoch [8/60], train_loss: 1.9164180358573795, train_accuracy: 0.9907294827252626, time: 13.356s
Epoch [9/60], train_loss: 1.6389805299937725, train_accuracy: 0.9163037165552378, time: 13.4s
Epoch [10/60], train_loss: 1.4107889053560794, train_accuracy: 0.8516696571409702, time: 13.416s
Epoch [11/60], train_loss: 1.2

In [12]:
test(test_dataset, BATCH_SIZE, model, nn.MSELoss(), accuracy)

Dataset size: 1117
test_loss: 2.163027650934303, test_accuracy: 0.9684234375172464, time: 0.498s


In [13]:
test(train_dataset, BATCH_SIZE, model, nn.MSELoss(), accuracy)

Dataset size: 8000
test_loss: 0.26817318103602156, test_accuracy: 0.37172700846008955, time: 3.351s


In [14]:
test(validation_dataset, BATCH_SIZE, model, nn.MSELoss(), accuracy)

Dataset size: 727
test_loss: 21.260763609097783, test_accuracy: 3.7825891942072767, time: 0.333s
